# Comparación de Embeddings Preentrenados de Hugging Face


En este notebook, utilizaremos varios modelos de embeddings preentrenados de Hugging Face. Nuestro objetivo es encontrar las palabras más similares a unas dadas utilizando diferentes modelos de embeddings.

**Modelos que utilizaremos:**
- BERT
- RoBERTa
- DistilBERT

Exploraremos cómo cada modelo interpreta las palabras y calcularemos las similitudes para encontrar las palabras más cercanas.
    

In [1]:
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity


## Cargar Modelos de Embeddings Preentrenados

In [16]:
# Definir los modelos a utilizar
models = {
    "BERT": "bert-base-uncased",
    "RoBERTa": "roberta-base",
    "DistilBERT": "distilbert-base-uncased"
}
models = {
    "BERT": "bert-base-uncased"
}

# Cargar los modelos y tokenizers
loaded_models = {name: AutoModel.from_pretrained(model) for name, model in models.items()}
tokenizers = {name: AutoTokenizer.from_pretrained(model) for name, model in models.items()}

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Extracción de Embeddings para Palabras Específicas

In [20]:
def get_word_embedding(word, model, tokenizer):
    # Tokenizar la palabra y convertirla en tensores
    inputs = tokenizer(word, return_tensors='pt')
    print(inputs)
    outputs = model(**inputs)
    # Extraer el embedding de la última capa oculta
    return outputs.last_hidden_state.squeeze().detach()

# Obtener los embeddings para las palabras "Almería" y "Banco"
words = ["Almería", "Banco"]
embeddings = {word: {} for word in words}

for word in words:
    for name, model in loaded_models.items():
        embedding = get_word_embedding(word, model, tokenizers[name])
        embeddings[word][name] = embedding
        print(f"Embedding obtenido para la palabra '{word}' con el modelo {name}.")
        print(f"El embedding tiene dimensiones: {embedding.shape}")

{'input_ids': tensor([[ 101, 2632, 5017, 2401,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
Embedding obtenido para la palabra 'Almería' con el modelo BERT.
El embedding tiene dimensiones: torch.Size([5, 768])
{'input_ids': tensor([[  101, 28678,   102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}
Embedding obtenido para la palabra 'Banco' con el modelo BERT.
El embedding tiene dimensiones: torch.Size([3, 768])


## Cálculo de Similitudes y Palabras Más Similares

In [ ]:
def find_most_similar_words(word_embedding, model_name, tokenizer, top_n=10):
    # Obtener todas las palabras del vocabulario del tokenizer
    vocab = list(tokenizer.get_vocab().keys())
    similarities = []

    # Calcular la similitud coseno entre la palabra dada y cada palabra del vocabulario
    for token in vocab:
        token_embedding = get_word_embedding(token, loaded_models[model_name], tokenizer)
        similarity = cosine_similarity([word_embedding.numpy()], [token_embedding.numpy()])[0][0]
        similarities.append((token, similarity))

    # Ordenar las palabras por similitud descendente y devolver las top_n palabras más similares
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Encontrar las palabras más similares para "Almería" y "Banco" con cada modelo
for word in words:
    print(f"Palabras más similares a '{word}':")
    for model_name in models.keys():
        similar_words = find_most_similar_words(embeddings[word][model_name], model_name, tokenizers[model_name])
        print(f"Modelo: {model_name}")
        for similar_word, score in similar_words:
            print(f"{similar_word}: {score:.4f}")
    print("="*50)

# Ejercicio

### Ejercicio 1
¿Por qué tarda tanto en obtener las palabras similares? Optimiza el código para que se ejecute mucho más rápido.
Pista: ¿Cada cuánto se ejecuta el modelo de obtención de embeddings?

Porque se van obteniendo los embeddings uno a uno, en lugar de haciendo uso de un batch de datos.

In [26]:
def get_word_batch_embedding(tokens, model, tokenizer):
    batch_size = 16  # Comienza con un tamaño de batch seguro
    max_sequence_length = 512  # Longitud máxima para la mayoría de los modelos como BERT o GPT-2
    print("Entra aqui")
    res = []
    # Dividir el texto en batches de tamaño adecuado
    for i in tqdm(range(0, len(tokens), batch_size * max_sequence_length)):
        batch = tokens[i:i + batch_size * max_sequence_length]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs)
            print("Se consulta el modelo")
            # Usar la salida de la última capa oculta como los embeddings
            embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # Usar solo el token [CLS] para cada secuencia
        for i in range(embeddings.shape[0]):
            res.append(embeddings[i])
    return res

In [ ]:
import numpy as np

def find_most_similar_words(word_embedding, model_name, tokenizer, top_n=10):
    # Obtener todas las palabras del vocabulario del tokenizer
    vocab = list(tokenizer.get_vocab().keys())
    similarities = []

    embeddings = get_word_batch_embedding(vocab, loaded_models[model_name], tokenizer)

    # Calcular la similitud coseno entre la palabra dada y cada palabra del vocabulario
    for t in range(len(vocab)):
        token = vocab[t]
        token_embedding = embeddings[t]

        if word_embedding.numpy().shape[0] > 1:
          # Hacer las 5 similitudes de coseno individuales
          cosine_sim_values = []
          for i in range(5):
              single_word_embedding = word_embedding[i, :]
              similarity = cosine_similarity([single_word_embedding], [token_embedding])[0][0]
              cosine_sim_values.append(similarity)
          # Calcular la media de las 5 similitudes
          similarity = np.mean(cosine_sim_values)
        else:
          similarity = cosine_similarity([word_embedding.numpy()], [token_embedding])[0][0]
        similarities.append((token, similarity))

    # Ordenar las palabras por similitud descendente y devolver las top_n palabras más similares
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return similarities[:top_n]


words = ["Almería", "Banco"]

# Encontrar las palabras más similares para "Almería" y "Banco" con cada modelo
for word in words:
    print(f"Palabras más similares a '{word}':")
    for model_name in models.keys():
        similar_words = find_most_similar_words(embeddings[word][model_name], model_name, tokenizers[model_name])
        print(f"Modelo: {model_name}")
        for similar_word, score in similar_words:
            print(f"{similar_word}: {score:.4f}")
    print("="*50)

```
Palabras más similares a 'Almería':
Entra aqui
 25%|██▌       | 1/4 [02:05<06:16, 125.40s/it]Se consulta el modelo
 50%|█████     | 2/4 [04:06<04:05, 122.93s/it]Se consulta el modelo
 75%|███████▌  | 3/4 [06:07<02:02, 122.12s/it]Se consulta el modelo
100%|██████████| 4/4 [07:37<00:00, 114.29s/it]Se consulta el modelo

Modelo: BERT
rihanna: 0.2528
immigration: 0.2498
botany: 0.2464
tq: 0.2462
cicero: 0.2460
plato: 0.2446
##eration: 0.2443
somme: 0.2443
cello: 0.2436
##erative: 0.2434
==================================================
```

### Ejercicio 2
Generar los embeddings para dos frases diferentes, que contengan la misma palabra. Muestra el embedding de dicha palabra. ¿Qué observas?

In [29]:
# Cargar el modelo preentrenado y el tokenizador
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Texto de entrada

texts = ["En Almería hace calor.", "En Almería no hace calor."]

for text in texts:
  # Tokenizar el texto
  inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True)

  # Pasar los inputs por el modelo para obtener los embeddings
  with torch.no_grad():
      outputs = model(**inputs)

  # Extraer los embeddings de la última capa
  last_hidden_state = outputs.last_hidden_state  # Tensor de tamaño (batch_size, sequence_length, hidden_size)

  # Obteniendo los embeddings de cada token
  token_embeddings = last_hidden_state.squeeze(0)  # Eliminar la dimensión del batch si solo hay un input
  print(token_embeddings.shape)
  # Mostramos los tokens y sus correspondientes embeddings
  tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())

  for token, embedding in zip(tokens, token_embeddings):
      print(f"Token: {token}, Embedding shape: {embedding.shape}, Primeros digitos = {embedding[0:2]}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


torch.Size([11, 768])
Token: [CLS], Embedding shape: torch.Size([768]), Primeros digitos = tensor([-0.4796,  0.0542])
Token: en, Embedding shape: torch.Size([768]), Primeros digitos = tensor([-0.0212, -0.1596])
Token: al, Embedding shape: torch.Size([768]), Primeros digitos = tensor([ 0.0306, -0.6478])
Token: ##mer, Embedding shape: torch.Size([768]), Primeros digitos = tensor([ 0.5880, -0.3254])
Token: ##ia, Embedding shape: torch.Size([768]), Primeros digitos = tensor([ 0.2525, -0.0318])
Token: ha, Embedding shape: torch.Size([768]), Primeros digitos = tensor([ 0.0630, -1.0317])
Token: ##ce, Embedding shape: torch.Size([768]), Primeros digitos = tensor([ 0.1877, -0.0969])
Token: cal, Embedding shape: torch.Size([768]), Primeros digitos = tensor([ 0.6961, -0.3533])
Token: ##or, Embedding shape: torch.Size([768]), Primeros digitos = tensor([-0.1220,  0.0452])
Token: ., Embedding shape: torch.Size([768]), Primeros digitos = tensor([-0.2790, -0.4968])
Token: [SEP], Embedding shape: torch